# Author John Payne
# Step 1
* Generate a response using the Casseopeia wrapper

In [1]:
import requests as re
import time

def getAPI_key():
    f = open("../api_key.txt", "r")
    return f.read()
api_key = getAPI_key()

In [2]:
def check_status():    
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

    return re.get(url, headers = header_dict).status_code

In [3]:
check_status()

200

In [4]:
def get_challenger_player_stats():
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }

    
    return re.get(url, headers = header_dict).json()

In [5]:
challenger_ladder = get_challenger_player_stats()

In [6]:
def get_challenger_player_id(player_id):
    
    #Using league_v4 
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + player_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

        
    return re.get(url, headers = header_dict).json()

In [7]:
def get_matches(puuid, startTime_Unix, endTime_Unix):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    params_dict = {
        #Only picking games played on patch 12.15, 08-10-2022 to 0-24-2022
        'startTime' : startTime_Unix,
        'endTime' : endTime_Unix,
        
        #Only picking ranked 5v5 games
        'queue' : 420,
        'type' : 'ranked',
        
        #Only grabbing the first 50 results
        'start' : 0,
        'count' : 50
    }
    return re.get(url, headers = header_dict, params = params_dict).json()

In [8]:
class Summoner:
    
    def __init__(self, summonerId, startTime_Unix, endTime_Unix):
        self.summonerId = summonerId
        
        #Looping through the ladder until a match is found with the input summoner id
        for value in challenger_ladder['entries']:
            if value['summonerId'] == self.summonerId:
                
                #Once correct player is found assign lp, wins, losses, winstreak, and ign to the player
                self.leaguepoints = value['leaguePoints']
                self.wins = value['wins']
                self.losses = value['losses']
                self.hotStreak = value['hotStreak']
                self.summonerName = value['summonerName']
        
        #Finds all the accounts of the player in dictionary format
        self.accounts = get_challenger_player_id(summonerId)
        #Finds the puuid of the player
        self.puuid = self.accounts['puuid']
        #Finds the players summoner level
        self.summonerLevel = self.accounts['summonerLevel']
        
        #Returns a list of matches the player has played based on the parameters in the get_matches function
        self.matches = get_matches(self.puuid, startTime_Unix, endTime_Unix)
        
        #Total games played by this summoner on this specific patch
        self.total_matches = len(self.matches)
        
    

In [9]:
player_info = {}
for i in range(2):
    temp = 'summoner' + str(i)
    player_info[temp] = Summoner(challenger_ladder['entries'][i]['summonerId'], 1660125600, 1661335200)
    print(i)
    time.sleep(1)

0
1


In [10]:
def get_match_info(match):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + match
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    return re.get(url, headers = header_dict).json()

In [21]:
class Match_details:
    
    def __init__(self, Match_id):
        self.Match_id = Match_id
        
        #Calling matchesV5 api
        self.call = get_match_info(Match_id)
        
        #Storing api call and filtering out uneeded data
        self.Match_info = self.call['info']
        
        #list of dictionaries including stats for each player
        self.players = self.Match_info['participants']
        
        #Tracking team information
        self.team1 = self.Match_info['teams'][0]
        self.team2 = self.Match_info['teams'][1]
        
        #Storing winning and losing team
        self.team1_win = self.team1['win']
        self.team2_win = self.team2['win']
        
        #Storing bans for each team
        self.team2_bans = self.team1['bans']
        self.team2_bans = self.team1['bans']
      
        #Creating a list of encrypted summoner ids
        self.summoner_Id = []
        for i, player in enumerate(self.players):
            self.summoner_Id.append(player['summonerId'])
        
        #Creating a player class to hold data on each player in the game
        class Player:
    
            def __init__(self, summoner_Id, players):
                self.summoner_Id = summoner_Id
                # Holds lane, champion and team of given player
                self.players = players
                for value in self.players:
                    if value['summonerId'] == self.summoner_Id:
                        self.Lane = value['individualPosition']
                        self.championId = value['championId'] 
                        self.teamId = value['teamId']
                        if self.teamId == 100:
                            self.teamId = 'team1'
                        elif self.teamId == 200:
                            self.teamId = 'team2'
                #Grabbing winrate/winstreak/rank/lp of the given player        
                self.player_stats = get_player_stats(self.summoner_Id)
        
        #Creating a dictionary to hold all the player information (lane, champion, team)
        self.player_position = {}
        for i, summoner_Id in enumerate(self.summoner_Id):
            temp = 'Player' + str(i)
            self.player_position[temp] = Player(summoner_Id, self.players)
            

In [12]:
#Getting matches for each player listed in player_info
games = []
for key in player_info:
    games.append(player_info[key].matches)

In [13]:
#Joining together the sublists (games of each player)
total_games = []
for a in range(len(games)):
    for b in range (len(games[a])):
        total_games.append(games[a][b])

In [14]:
#Finding unique games
total_games = list(set(total_games))

In [15]:
#API request to get player winrates etc.
def get_player_stats(summoner_id):
    url = 'https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summoner_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    return re.get(url, headers = header_dict).json()

In [36]:
%%time
games = {}
for i, game in enumerate(total_games):  
    temp = 'Game' + str(i)
    games[temp] = Match_details(game)
    print(i)
    time.sleep(10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
CPU times: total: 3.92 s
Wall time: 6min 38s


In [660]:
#What info do i want in my final dataframe?

# Match Id, team1 bans, team1 picks, team1 players, team1Winrates, team2 bans, team2 picks, team2 players, winner (team1 or team2)
# Json example
x = {'Game_info' : [{'Team1' : [
    {'bans' : 
    [{'Player1' : 'Wukong'}, 
    {'Player2' : 'Graves'}, 
    {'Player3' : 'Azir'}, 
    {'Player4' : 'Vayne'}, 
    {'Player5' : 'Braum'}]},
    {'picks' : 
    [{'Player1' : 'Aatrox'}, 
    {'Player2' : 'Shyvana'}, 
    {'Player3' : 'Orianna'}, 
    {'Player4' : 'Caitlyn'}, 
    {'Player5' : 'Lux'}]},
    {'PlayerNames' :
    [{'Player1' : 'TopLaneDude'}, 
    {'Player2' : 'Jungleplayer123'}, 
    {'Player3' : 'JungleDiff100%'}, 
    {'Player4' : 'ADCfarmOnly'}, 
    {'Player5' : 'NoobEnchanter'}]},
    {'PlayerWinrate' :
    [{'Player1' : '.52'}, 
    {'Player2' : '.63'}, 
    {'Player3' : '.49'}, 
    {'Player4' : '.55'}, 
    {'Player5' : '.51'}]}]},
     
    {'Team2' : [
    {'bans' : 
    [{'Player1' : 'Wukong'}, 
    {'Player2' : 'Graves'}, 
    {'Player3' : 'Azir'}, 
    {'Player4' : 'Vayne'}, 
    {'Player5' : 'Braum'}]},
    {'picks' : 
    [{'Player1' : 'Aatrox'}, 
    {'Player2' : 'Shyvana'}, 
    {'Player3' : 'Orianna'}, 
    {'Player4' : 'Caitlyn'}, 
    {'Player5' : 'Lux'}]},
    {'PlayerNames' :
    [{'Player1' : 'TopLaneDude'}, 
    {'Player2' : 'Jungleplayer123'}, 
    {'Player3' : 'JungleDiff100%'}, 
    {'Player4' : 'ADCfarmOnly'}, 
    {'Player5' : 'NoobEnchanter'}]},
    {'PlayerWinrate' :
    [{'Player1' : '.52'}, 
    {'Player2' : '.63'}, 
    {'Player3' : '.49'}, 
    {'Player4' : '.55'}, 
    {'Player5' : '.51'}]}]},
    {'Winner' : 'Team1'}
    ]}
    
 

In [ ]:
# Column format: Match ID, Player1 ign,, Player 2 ign, Player 3 ign, Player 4 ign, Player 5 ign, Player 6 ign, Player 7 ign, Player 8 ign, Player 9 ign, Player 10 ign
# Player1 champion, Player 2 champion, Player 3 champion, Player 4 champion, Player 5 champion, Player 6 champion, Player 7 champion, Player 8 champion, Player 9 champion, Player 10 champion
# Player1 role, Player 2 role, Player 3 role, Player 4 role, Player 5 role, Player 6 role, Player 7 role, Player 8 role, Player 9 role, Player 10 role
# Player1 lp, Player 2 lp, Player 3 lp, Player 4 lp, Player 5 lp, Player 6 lp, Player 7 lp, Player 8 lp, Player 9 lp, Player 10 lp
# Player1 ban, Player 2 ban, Player 3 ban, Player 4 ban, Player 5 ban, Player 6 ban, Player 7 ban, Player 8 ban, Player 9 ban, Player 10 ban
# Player1 winrate, Player 2 winrate, Player 3 winrate, Player 4 winrate, Player 5 winrate, Player 6 winrate, Player 7 winrate, Player 8 winrate, Player 9 winrate, Player 10 winrate
# Player1 total_games, Player 2 total_games, Player 3 total_games, Player 4 total_games, Player 5 total_games, Player 6 total_games, Player 7 total_games, Player 8 total_games, Player 9 total_games, Player 10 total_games

#Starting features
#Team1 total lp
#Team2 total lp

Match_details = []
